In [ ]:
# %pip install cython
# %pip install sortedcontainers
# %pip install zipp==3.1.0 --upgrade
# %pip install scorer
# %pip install -r re

In [ ]:
import os
import sys
from pathlib import Path

# Если запускаемся на DataSphere
if str(Path.home()) == "/home/jupyter":
    os.chdir("/home/jupyter/work/resources/wunder_summer/examples/datasphere/catboost")
    sys.path.append("/home/jupyter/work/resources/wunder_summer/scorer/orderbook_fast")
    sys.path.append("/home/jupyter/work/resources/wunder_summer/scorer")
    sys.path.append("/home/jupyter/work/resources/wunder_summer/examples/catboost_local")
    data_dir = Path("/home/jupyter/work/resources/wunder_summer/data")
print(Path.cwd())

In [ ]:
# %pip install -r datasphere_requirements.txt --upgrade
# %pip install -r /home/jupyter/work/resources/wunder_summer/requirements.txt --upgrade
# %pip install -r /home/jupyter/work/resources/wunder_summer/scorer_packages.txt --upgrade

In [ ]:
import pyximport
# import scorer
import pystan
import numpy as np
import pandas as pd
# import lightgbm as lg
import catboost
import zipp
from catboost.utils import get_gpu_device_count
from tqdm import tqdm
from pathlib import Path

import orderbook_fast as ob
from my_orderbook import MyOrderBook

print("curdir = ", Path.cwd())
print("homedir = ", Path.home())




SIDE_BID = 0
SIDE_ASK = 1

## Импортируем датасет для тренировки модели

In [ ]:
def train_classifier(X_train, Y_train, X_test, Y_test):
    '''
        Обучаем классификатор
    '''
    clf = lg.LGBMClassifier(num_leaves=31, n_estimators=1000, learning_rate=0.1)
    clf.fit(X_train, Y_train, eval_set=[(X_test, Y_test)],
            eval_metric="auc", early_stopping_rounds=20)
    return clf





## Обучаем модель градиентного бустинга

In [ ]:
#!L

X_train = pd.read_pickle(data_dir/"X_train80.pickle")
Y_train = pd.read_pickle("/home/jupyter/work/resources/wunder_summer/data/Y_train80.pickle")

X_test = pd.read_pickle("/home/jupyter/work/resources/wunder_summer/data/X_test20.pickle")
Y_test = pd.read_pickle("/home/jupyter/work/resources/wunder_summer/data/Y_test20.pickle")

print("X_train.shape", X_train.shape)
print("Y_test.shape", Y_test.shape)

clf = train_classifier(X_train, Y_train, X_test, Y_test)

# Сохраним нашу модель
os.chdir("/home/jupyter/work/resources/wunder_summer/examples/datasphere/catboost")
sys.path.append("/home/jupyter/work/resources/wunder_summer/scorer/orderbook_fast")
sys.path.append("/home/jupyter/work/resources/wunder_summer/scorer")
sys.path.append("/home/jupyter/work/resources/wunder_summer/examples/catboost_local")

print(Path.cwd())
clf.save_model("wunder.model", format="cbm")


## Тестируем получившийся классификатор

In [ ]:
# import pystan

In [ ]:
# %pip install cmake
# %pip install scorer==2.0.2 --upgrade --use-feature=2020-resolver --force-reinstall
# %pip install pystan


In [ ]:
from my_orderbook import MyOrderBook, action_handler

lightgbm_myob = MyOrderBook()

def process_event_and_predict_proba(ev, orderbook):
    lightgbm_myob.__getattribute__(action_handler[ev.action])

    if not ev.need_prediction:
        return None

    features = lightgbm_myob.get_features(ev, orderbook, max_index=13)
    proba = clf.predict_proba([features])[0, 1]
    return proba

from scorer import Scorer

scoring = Scorer("../../../data/train_small_C.npz")
# scoring = Scorer("../../../data/very_small_B.npz")
roc_auc, (true_ys, pred_probas) = scoring.score(process_event_and_predict_proba)

In [ ]:
## Сохраним модель, и решение для отправки готово

## Сохраним модель, и решение для отправки готово

In [ ]:
# Сохраним нашу модель
clf.booster_.save_model("wunder.model")
print(Path.cwd())
# clf.save_model("wunder.model", format="cbm")

'''Посмотрите код файла solution.py.
Он использует те же функции что и этот ноутбук, но уже готов к отправке на серверю.
Попробуйте создать архив с файлами solution.py и wunder.model и отправить их на проверку.'''

In [ ]:
#!S
